In [1]:
%matplotlib inline
import wget
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Read + Clean Up Australian Open

In [2]:
df = pd.read_csv("./data/tournament/atp_tennis_2025.csv")
df

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Jasika O.,Gaston H.,Gaston H.,179,81,323,703,3.20,1.36,2-6 6-3 2-6 2-6
1,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Fils A.,Virtanen O.,Fils A.,21,93,2280,627,1.13,6.00,3-6 7-6 6-4 6-4
2,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Monteiro T.,Nishikori K.,Nishikori K.,105,76,566,743,4.50,1.20,6-4 7-6 5-7 2-6 3-6
3,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Ruud C.,Munar J.,Ruud C.,6,62,4210,922,1.17,5.00,6-3 1-6 7-5 2-6 6-1
4,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Bu Y.,Habib H.,Habib H.,67,219,784,264,1.36,3.20,6-7 4-6 6-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Rio Open,2025-02-21,ATP500,Outdoor,Clay,Quarterfinals,3,Comesana F.,Zverev A.,Comesana F.,86,2,673,8135,6.00,1.13,4-6 6-3 6-4
330,Rio Open,2025-02-22,ATP500,Outdoor,Clay,Quarterfinals,3,Cerundolo F.,Muller A.,Muller A.,26,60,1825,953,1.50,2.63,5-7 1-6
331,Rio Open,2025-02-22,ATP500,Outdoor,Clay,Semifinals,3,Baez S.,Ugo Carabelli C.,Baez S.,31,91,1510,644,1.36,3.20,3-6 6-1 6-1
332,Rio Open,2025-02-22,ATP500,Outdoor,Clay,Semifinals,3,Comesana F.,Muller A.,Muller A.,86,60,673,953,2.50,1.53,5-7 7-6 3-6


In [3]:
aus_open = df[df["Tournament"] == "Australian Open"]
aus_open

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Jasika O.,Gaston H.,Gaston H.,179,81,323,703,3.20,1.36,2-6 6-3 2-6 2-6
1,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Fils A.,Virtanen O.,Fils A.,21,93,2280,627,1.13,6.00,3-6 7-6 6-4 6-4
2,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Monteiro T.,Nishikori K.,Nishikori K.,105,76,566,743,4.50,1.20,6-4 7-6 5-7 2-6 3-6
3,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Ruud C.,Munar J.,Ruud C.,6,62,4210,922,1.17,5.00,6-3 1-6 7-5 2-6 6-1
4,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Bu Y.,Habib H.,Habib H.,67,219,784,264,1.36,3.20,6-7 4-6 6-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Australian Open,2025-01-21,Grand Slam,Outdoor,Hard,Quarterfinals,5,Djokovic N.,Alcaraz C.,Djokovic N.,7,3,3900,7010,2.50,1.53,4-6 6-4 6-3 6-4
112,Australian Open,2025-01-22,Grand Slam,Outdoor,Hard,Quarterfinals,5,Sonego L.,Shelton B.,Shelton B.,55,20,1026,2280,3.20,1.36,4-6 5-7 6-4 6-7
113,Australian Open,2025-01-22,Grand Slam,Outdoor,Hard,Quarterfinals,5,Sinner J.,De Minaur A.,Sinner J.,1,8,11830,3535,1.17,5.00,6-3 6-2 6-1
114,Australian Open,2025-01-24,Grand Slam,Outdoor,Hard,Semifinals,5,Shelton B.,Sinner J.,Sinner J.,20,1,2280,11830,11.00,1.05,6-7 2-6 2-6


In [4]:
aus_open_players = list(set(aus_open["Player_1"].to_list() + aus_open["Player_2"].to_list() + aus_open["Winner"].to_list()))
aus_open_players

['Hanfmann Y.',
 'Nakashima B.',
 'Passaro F.',
 'Bu Y.',
 'Nardi L.',
 'Medvedev D.',
 'Michelsen A.',
 'Machac T.',
 'Mannarino A.',
 'Thompson J.',
 'Auger-Aliassime F.',
 'Navone M.',
 'Habib H.',
 'Bublik A.',
 'Van De Zandschulp B.',
 'Ruud C.',
 'Lajovic D.',
 'Dougaz A.',
 'Borges N.',
 'Kokkinakis T.',
 'Cobolli F.',
 'Griekspoor T.',
 'Goffin D.',
 'Alcaraz C.',
 'Coria F.',
 'Djokovic N.',
 'Carballes Baena R.',
 'Landaluce M.',
 'Duckworth J.',
 'Gigante M.',
 'Ugo Carabelli C.',
 'Fearnley J.',
 'Diallo G.',
 'Fils A.',
 'Halys Q.',
 'Virtanen O.',
 'Rinderknech A.',
 'Mensik J.',
 'Rublev A.',
 'Struff J.L.',
 'Gaston H.',
 'Popyrin A.',
 'Kotov P.',
 'Fonseca J.',
 'Krueger M.',
 'Munar J.',
 'Stricker D.',
 'Vukic A.',
 'Nagal S.',
 'Bautista Agut R.',
 'Fritz T.',
 'Khachanov K.',
 'Basilashvili N.',
 'Hurkacz H.',
 'Tiafoe F.',
 'Draper J.',
 'Musetti L.',
 'Samrej K.',
 'Jasika O.',
 'Hijikata R.',
 'Bergs Z.',
 'Zverev A.',
 'Onclin G.',
 'Jarry N.',
 'Garin C.',
 '

In [5]:
len(aus_open_players)

123

In [6]:
players = pd.read_csv("./data/atp_players.csv")
players

/var/folders/2n/3gvc3mg10j76nnrf7fqlxh640000gn/T/ipykernel_32480/652814457.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("./data/atp_players.csv")


,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554
...,...,...,...,...,...,...,...,...
65984,213700,Matvei,Kobiakov,U,NaN,RUS,NaN,NaN
65985,213701,Tobia Costanzo,Baragiola Mordini,U,NaN,ITA,NaN,NaN
65986,213702,Dominik,Wijntjes,U,NaN,NZL,NaN,NaN
65987,213703,Sam,Wensley,U,NaN,AUS,NaN,NaN


In [7]:
# Create dictionary
mappings = {}

for player in aus_open_players:
    player_d = player.replace(".", "")
    last_name = player_d[:-2].strip()
    first_i = player_d[-1]
    
    p = players[(players["name_last"] == last_name) & (players["name_first"].str.startswith(first_i))]
    p = p.dropna(subset=["player_id", "name_first", "name_last", "hand", "dob", "ioc", "height"])

    if len(p) == 1:
        mappings[player] = p["player_id"].to_list()[0]
    else:
        print(player)
    
    

Bu Y.
Auger-Aliassime F.
Ruud C.
Landaluce M.
Struff J.L.
Popyrin A.
Zhang Zh.
Mpetshi G.
Shelton B.
Paul T.
O Connell C.
Boyer T.


In [8]:
mappings["O Connell C."] = 106331
mappings["Landaluce M."] = 211776
mappings["Boyer T."] = 207729
mappings["Bu Y."] = 207352
mappings["Mpetshi G."] = 208659
mappings["Struff J.L."] = 105526
mappings["Ruud C."] = 134770
mappings["Auger-Aliassime F."] = 200000
mappings["Paul T."] = 126205
mappings["Popyrin A."] = 200615
mappings["Shelton B."] = 210097
mappings["Zhang Zh."] = 111190

In [9]:
players_dict = players.set_index("player_id").to_dict(orient="index")
players_dict

{100001: {'name_first': 'Gardnar',
  'name_last': 'Mulloy',
  'hand': 'R',
  'dob': 19131122.0,
  'ioc': 'USA',
  'height': 185.0,
  'wikidata_id': 'Q54544'},
 100002: {'name_first': 'Pancho',
  'name_last': 'Segura',
  'hand': 'R',
  'dob': 19210620.0,
  'ioc': 'ECU',
  'height': 168.0,
  'wikidata_id': 'Q54581'},
 100003: {'name_first': 'Frank',
  'name_last': 'Sedgman',
  'hand': 'R',
  'dob': 19271002.0,
  'ioc': 'AUS',
  'height': 180.0,
  'wikidata_id': 'Q962049'},
 100004: {'name_first': 'Giuseppe',
  'name_last': 'Merlo',
  'hand': 'R',
  'dob': 19271011.0,
  'ioc': 'ITA',
  'height': nan,
  'wikidata_id': 'Q1258752'},
 100005: {'name_first': 'Richard',
  'name_last': 'Gonzalez',
  'hand': 'R',
  'dob': 19280509.0,
  'ioc': 'USA',
  'height': 188.0,
  'wikidata_id': 'Q53554'},
 100006: {'name_first': 'Grant',
  'name_last': 'Golden',
  'hand': 'R',
  'dob': 19290821.0,
  'ioc': 'USA',
  'height': 175.0,
  'wikidata_id': 'Q3115390'},
 100007: {'name_first': 'Abe',
  'name_last':

## Create Dataset

In [10]:
aus_open

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Jasika O.,Gaston H.,Gaston H.,179,81,323,703,3.20,1.36,2-6 6-3 2-6 2-6
1,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Fils A.,Virtanen O.,Fils A.,21,93,2280,627,1.13,6.00,3-6 7-6 6-4 6-4
2,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Monteiro T.,Nishikori K.,Nishikori K.,105,76,566,743,4.50,1.20,6-4 7-6 5-7 2-6 3-6
3,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Ruud C.,Munar J.,Ruud C.,6,62,4210,922,1.17,5.00,6-3 1-6 7-5 2-6 6-1
4,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Bu Y.,Habib H.,Habib H.,67,219,784,264,1.36,3.20,6-7 4-6 6-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Australian Open,2025-01-21,Grand Slam,Outdoor,Hard,Quarterfinals,5,Djokovic N.,Alcaraz C.,Djokovic N.,7,3,3900,7010,2.50,1.53,4-6 6-4 6-3 6-4
112,Australian Open,2025-01-22,Grand Slam,Outdoor,Hard,Quarterfinals,5,Sonego L.,Shelton B.,Shelton B.,55,20,1026,2280,3.20,1.36,4-6 5-7 6-4 6-7
113,Australian Open,2025-01-22,Grand Slam,Outdoor,Hard,Quarterfinals,5,Sinner J.,De Minaur A.,Sinner J.,1,8,11830,3535,1.17,5.00,6-3 6-2 6-1
114,Australian Open,2025-01-24,Grand Slam,Outdoor,Hard,Semifinals,5,Shelton B.,Sinner J.,Sinner J.,20,1,2280,11830,11.00,1.05,6-7 2-6 2-6


In [11]:
from datetime import datetime

def calculate_age(dob):
    dob_str = str(int(dob))
    dob_date = datetime.strptime(dob_str, "%Y%m%d")
    today = datetime.today()
    age_years = today.year - dob_date.year

    dob_month_day = dob_date.replace(year=today.year)
    if today < dob_month_day:
        age_years -= 1 

    days_in_year = 365.25
    days_since_birthday = (today - dob_month_day).days if today >= dob_month_day else (today - dob_month_day.replace(year=today.year - 1)).days
    age_fractional = days_since_birthday / days_in_year
    
    return age_years + age_fractional


In [12]:
winner_id = []
loser_id = []
winner_rank_points = []
loser_rank_points = []
winner_rank = []
loser_rank = []
winner_age = []
loser_age = []
winner_ht = []
loser_ht = []

for idx, (p1, p2, w1, r1, r2, pts1, pts2) in enumerate(tqdm(zip(aus_open['Player_1'], aus_open['Player_2'], aus_open['Winner'], 
                                                                aus_open['Rank_1'], aus_open['Rank_2'],
                                                                aus_open['Pts_1'], aus_open['Pts_2'],
                                                                ),total=len(aus_open))):
    winner_id.append(mappings[w1])

    if p1 == w1:
        winner = p1
        loser = p2

        winner_rank.append(r1)
        loser_rank.append(r2)
        winner_rank_points.append(pts1)
        loser_rank_points.append(pts2)

    elif p2 == w1:
        winner = p2
        loser = p1

        winner_rank.append(r2)
        loser_rank.append(r1)
        winner_rank_points.append(pts2)
        loser_rank_points.append(pts1)

    else:
        raise Exception()
    
    loser_id.append(mappings[loser])
    winner_ht.append(players_dict[mappings[winner]]["height"])
    loser_ht.append(players_dict[mappings[loser]]["height"])
    winner_age.append(calculate_age(players_dict[mappings[winner]]["dob"]))
    loser_age.append(calculate_age(players_dict[mappings[loser]]["dob"]))




100%|██████████| 116/116 [00:00<00:00, 47560.05it/s]


## Final AUS Open Dataset

In [13]:
columns = [
    "tourney_id", "tourney_name", "surface", "draw_size", "tourney_level", "tourney_date", 
    "match_num", "winner_id", "winner_seed", "winner_entry", "winner_name", "winner_hand", 
    "winner_ht", "winner_ioc", "winner_age", "loser_id", "loser_seed", "loser_entry", 
    "loser_name", "loser_hand", "loser_ht", "loser_ioc", "loser_age", "score", "best_of", 
    "round", "minutes", "w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", 
    "w_SvGms", "w_bpSaved", "w_bpFaced", "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", 
    "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced", "winner_rank", "winner_rank_points", 
    "loser_rank", "loser_rank_points"
]

# Create a DataFrame with zeros
final_dataset_aus_open = pd.DataFrame(np.zeros((len(aus_open), len(columns))), columns=columns)
final_dataset_aus_open

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
final_dataset_aus_open["winner_id"] = winner_id
final_dataset_aus_open["winner_id"] = winner_id
final_dataset_aus_open["loser_id"] = loser_id
final_dataset_aus_open["winner_rank_points"] = winner_rank_points
final_dataset_aus_open["loser_rank_points"] = loser_rank_points
final_dataset_aus_open["winner_rank"] = winner_rank
final_dataset_aus_open["loser_rank"] = loser_rank
final_dataset_aus_open["winner_age"] = winner_age
final_dataset_aus_open["loser_age"] = loser_age
final_dataset_aus_open["winner_ht"] = winner_ht
final_dataset_aus_open["loser_ht"] = loser_ht
final_dataset_aus_open["best_of"] = 5
final_dataset_aus_open["draw_size"] = 128
final_dataset_aus_open["surface"] = "Hard"
final_dataset_aus_open["tourney_id"] = "AUSTRALIAN_OPEN_2025"

In [15]:
final_dataset_aus_open.to_csv("./data/all/aus_open_2025.csv")

In [16]:
final_dataset_aus_open

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,200384,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,81,703,179,323
1,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,209950,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,21,2280,93,627
2,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,105453,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,76,743,105,566
3,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,134770,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,4210,62,922
4,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,200273,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,219,264,67,784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,104925,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7,3900,3,7010
112,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,210097,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20,2280,55,1026
113,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,206173,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,11830,8,3535
114,AUSTRALIAN_OPEN_2025,0.0,Hard,128,0.0,0.0,0.0,206173,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,11830,20,2280


In [17]:
aus_open

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Jasika O.,Gaston H.,Gaston H.,179,81,323,703,3.20,1.36,2-6 6-3 2-6 2-6
1,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Fils A.,Virtanen O.,Fils A.,21,93,2280,627,1.13,6.00,3-6 7-6 6-4 6-4
2,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Monteiro T.,Nishikori K.,Nishikori K.,105,76,566,743,4.50,1.20,6-4 7-6 5-7 2-6 3-6
3,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Ruud C.,Munar J.,Ruud C.,6,62,4210,922,1.17,5.00,6-3 1-6 7-5 2-6 6-1
4,Australian Open,2025-01-12,Grand Slam,Outdoor,Hard,1st Round,5,Bu Y.,Habib H.,Habib H.,67,219,784,264,1.36,3.20,6-7 4-6 6-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Australian Open,2025-01-21,Grand Slam,Outdoor,Hard,Quarterfinals,5,Djokovic N.,Alcaraz C.,Djokovic N.,7,3,3900,7010,2.50,1.53,4-6 6-4 6-3 6-4
112,Australian Open,2025-01-22,Grand Slam,Outdoor,Hard,Quarterfinals,5,Sonego L.,Shelton B.,Shelton B.,55,20,1026,2280,3.20,1.36,4-6 5-7 6-4 6-7
113,Australian Open,2025-01-22,Grand Slam,Outdoor,Hard,Quarterfinals,5,Sinner J.,De Minaur A.,Sinner J.,1,8,11830,3535,1.17,5.00,6-3 6-2 6-1
114,Australian Open,2025-01-24,Grand Slam,Outdoor,Hard,Semifinals,5,Shelton B.,Sinner J.,Sinner J.,20,1,2280,11830,11.00,1.05,6-7 2-6 2-6
